In [1]:
import pandas as pd
import time
from matplotlib import pyplot as plt
from datetime import datetime

In [2]:
def save(df):
    ts = int(time.time())
    df.to_excel("./files/{}.xlsx".format(ts))
    print(str(ts)+".xlsx has been saved")

In [3]:
df = pd.read_csv("./data/sample.csv",delimiter=",",dtype=str)
print(df.shape)

(21098, 33)


In [ ]:
!pip install bertopic

  Using cached bertopic-0.15.0-py2.py3-none-any.whl (143 kB)
  Using cached umap_learn-0.5.4-py3-none-any.whl
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached torchvision-0.16.0-cp310-cp310-manylinux1_x86_64.whl (6.9 MB)
  Using cached tbb-2021.10.0-py2.py3-none-manylinux1_x86_64.whl (4.4 MB)
  Using cached pynndescent-0.5.10-py3-none-any.whl
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Usin